In [21]:
import numpy as np
import cv2
import os
import pandas as pd
import string
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from PIL import Image

In [22]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [23]:
images = []
labels = []

RECORDS_COUNT = 115339

In [24]:
train_images = []
train_labels = []
train_input_length = []
train_label_length = []
train_original_text = []

valid_images = []
valid_labels = []
valid_input_length = []
valid_label_length = []
valid_original_text = []

inputs_length = []
labels_length = []

In [25]:
engl=pd.read_csv('D:/word_segmentation-master/Training/Img/english.csv')
engl.head()

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0


In [26]:
with open('D:/word_segmentation-master/IAM/IAM/words.txt') as f:
    contents = f.readlines()[18:115339]

lines = [line.strip() for line in contents]
lines[-1]

'r06-143-04-10 ok 186 1892 1453 36 66 ? ?'

In [28]:
def process_image(img):
    """
    Converts image to shape (32, 128, 1) & normalize
    """
    w, h = img.shape

#     _, img = cv2.threshold(img,
#                            128,
#                            255,
#                            cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Aspect Ratio Calculation
    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape

    img = img.astype('float32')

    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)

    img = cv2.subtract(255, img)

    img = np.expand_dims(img, axis=2)

    # Normalize
    img = img / 255

    return img

In [29]:
max_label_len = 0

char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

# string.ascii_letters + string.digits (Chars & Digits)
# or
# "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

print(char_list, len(char_list))

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, chara in enumerate(txt):
        dig_lst.append(char_list.index(chara))

    return dig_lst

!"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 78


In [30]:
for i in range(len(engl)):
     
        splits_id=engl["image"][i].split('/')
        filepath = 'D:/word_segmentation-master/Training/{}/{}'.format(splits_id[0], 
                                                  splits_id[1])
       
        # process image
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        try:
            img = process_image(img)
        except:
            continue
            
        # process label
        try:
            
            word=engl['label'][i]
            label = encode_to_labels(word)
        except:
            continue
        
        if i % 10 == 0:
            valid_images.append(img)
            valid_labels.append(label)
            valid_input_length.append(31)
            valid_label_length.append(len(word))
            valid_original_text.append(word)
        else:
            train_images.append(img)
            train_labels.append(label)
            train_input_length.append(31)
            train_label_length.append(len(word))
            train_original_text.append(word)
        
        if len(word) > max_label_len:
            max_label_len = len(word)
    
        if i >= RECORDS_COUNT:
            break

In [ ]:
splits_id[0]

In [ ]:
splits_id[1]

In [ ]:
lines

In [ ]:
splits = lines[0].split(' ')

In [ ]:
splits

In [ ]:
splits[1]

In [ ]:
for index, line in enumerate(lines):
    splits = line.split(' ')
    status = splits[1]

    if status == 'ok':
        word_id = splits[0]
        word = "".join(splits[8:])

        splits_id = word_id.split('-')
        filepath = 'D:/word_segmentation-master/IAM/words/{}/{}-{}/{}.png'.format(splits_id[0],
                                                  splits_id[0],
                                                  splits_id[1],
                                                  word_id)

        # process image
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        try:
            img = process_image(img)
        except:
            continue

        # process label
        try:
            label = encode_to_labels(word)
        except:
            continue

        if index % 10 == 0:
            valid_images.append(img)
            valid_labels.append(label)
            valid_input_length.append(31)
            valid_label_length.append(len(word))
            valid_original_text.append(word)
        else:
            train_images.append(img)
            train_labels.append(label)
            train_input_length.append(31)
            train_label_length.append(len(word))
            train_original_text.append(word)

        if len(word) > max_label_len:
            max_label_len = len(word)

    if index >= RECORDS_COUNT:
        break

In [ ]:

train_padded_label = pad_sequences(train_labels,
                             maxlen=max_label_len,
                             padding='post',
                             value=len(char_list))

valid_padded_label = pad_sequences(valid_labels,
                             maxlen=max_label_len,
                             padding='post',
                             value=len(char_list))

In [ ]:
train_labels[3101]

In [ ]:
train_padded_label[3101]

In [ ]:
train_padded_label.shape, valid_padded_label.shape

In [ ]:
train_images = np.asarray(train_images)
train_input_length = np.asarray(train_input_length)
train_label_length = np.asarray(train_label_length)

valid_images = np.asarray(valid_images)
valid_input_length = np.asarray(valid_input_length)
valid_label_length = np.asarray(valid_label_length)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as tf_keras_backend
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

tf_keras_backend.set_image_data_format('channels_last')
tf_keras_backend.image_data_format()

In [ ]:
def Model1():
    # input with shape of height=32 and width=128 
    inputs = Input(shape=(32,128,1))

    # convolution layer with kernel size (3,3)
    conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
    # poolig layer with kernel size (2,2)
    pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

    conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

    conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)

    conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
    # poolig layer with kernel size (2,1)
    pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

    conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
    # Batch normalization layer
    batch_norm_5 = BatchNormalization()(conv_5)

    conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

    conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

    squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

    # bidirectional LSTM layers with units=128
    blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(squeezed)
    blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(blstm_1)

    outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

    # model to be used at test time
    act_model = Model(inputs, outputs)
    
    return act_model,outputs,inputs

In [ ]:
import keras.backend as K

In [ ]:
act_model,outputs,inputs=Model1()

In [ ]:
act_model.summary()

In [ ]:
import os
import cv2
import numpy as np

# Define the directory where your test images are stored
test_data_dir = r'D:\word_segmentation-master\IAM\IAM\words\a01\a01-000u'

def load_test_data():
    test_images = []
    test_labels = []
    
    image_files = os.listdir(test_data_dir)
    
    for image_file in image_files:
        image_path = os.path.join(test_data_dir, image_file)
        
        # Assume the parent directory name (a01) is the class label
        class_label = os.path.basename(os.path.dirname(image_path))
        
        try:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            if image is not None:
                image = cv2.resize(image, (128, 32))  # Resize image to match your model's input shape
                test_images.append(image)
                test_labels.append(class_label)
            else:
                print(f"Failed to load image: {image_path}")
        except Exception as e:
            print(f"Error processing image: {image_path}, {str(e)}")
    
    # Convert the lists to numpy arrays
    test_images = np.array(test_images, dtype=np.float32) / 255.0  # Normalize pixel values
    test_labels = np.array(test_labels)
    
    return test_images, test_labels

# Load test data and labels
X_test, y_true = load_test_data()

# Now you can proceed to make predictions using your model and compute the confusion matrix as previously shown.


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Assuming you have already trained and compiled your model
# Load your test data
X_test, y_true = load_test_data()  # Replace with your actual data loading code

# Make predictions using your model
y_pred = act_model.predict(X_test)

# Convert the one-hot encoded predictions and true labels back to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_true, axis=1)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(y_true_classes, y_pred_classes)

# Print the confusion matrix
print(confusion_mtx)


In [ ]:
the_labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, the_labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, the_labels, input_length, label_length], outputs=loss_out)

In [ ]:
batch_size = 32
epochs = 50
e = str(epochs)
optimizer_name = 'sgd'

In [ ]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = optimizer_name, metrics=['accuracy'])

filepath="{}o-{}r-{}e-{}t-{}v.hdf5".format(optimizer_name,
                                          str(RECORDS_COUNT),
                                          str(epochs),
                                          str(train_images.shape[0]),
                                          str(valid_images.shape[0]))

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]


In [ ]:
history = model.fit(x=[train_images, train_padded_label, train_input_length, train_label_length],
                    y=np.zeros(len(train_images)),
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=([valid_images, valid_padded_label, valid_input_length, valid_label_length], [np.zeros(len(valid_images))]),
                    verbose=1,
                    callbacks=callbacks_list)


In [ ]:
model.save(filepath='./model3.h5', overwrite=False, include_optimizer=True)

In [ ]:
import Levenshtein as lv
filepath='D:/word_segmentation-master/sgdo-115339r-50e-89887t-9977v.hdf5'
# load the saved best model weights
act_model.load_weights(filepath)

# predict outputs on validation images
prediction = act_model.predict(valid_images)

# use CTC decoder
decoded = K.ctc_decode(prediction,
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]
out = K.get_value(decoded)



total_jaro = 0

# see the results
for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]
    total_jaro+=lv.jaro(letters, valid_original_text[i])

print('jaro :', total_jaro/len(out))


In [ ]:
# predict outputs on validation images
i=2000
j=2005
prediction = act_model.predict(valid_images[i:j])

# use CTC decoder
decoded = K.ctc_decode(prediction,
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]

out = K.get_value(decoded)

# see the results
for _, x in enumerate(out):

    print("original_text =  ", valid_original_text[i])
    print("predicted text = ", end = '')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    plt.imshow(valid_images[i].reshape(32,128), cmap=plt.cm.gray)
    plt.show()
    i+=1
    print('\n')

In [ ]:
#plot accuracy and loss
def plotgraph(epochs, acc, val_acc):
    # Plot training & validation accuracy values
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)


In [ ]:
import cv2
import numpy as np
# Load your image from file
for i in range(4):
 your_image_path = "D:\word_segmentation-master\segmented\segment"+ str(i) + ".png"
 your_image = cv2.imread(your_image_path, cv2.IMREAD_GRAYSCALE)

# Process the image using the process_image function
 processed_image = process_image(your_image)

# Expand dimensions to match the model's input shape
 processed_image = np.expand_dims(processed_image, axis=0)

# Make predictions on your processed image
 prediction = act_model.predict(processed_image)

# Use CTC decoder
 decoded = K.ctc_decode(prediction,
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]

 out = K.get_value(decoded)

# Display the results
 for _, x in enumerate(out):
    print("predicted text = ", end='')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end='')
    
    # Display your processed image
    plt.imshow(your_image, cmap=plt.cm.gray)
    plt.show()

      


In [ ]:
import cv2
import numpy as np
# Load your image from file
for i in range(4):
 your_image_path = "D:\word_segmentation-master\segmented\segment"+ str(i) + ".png"
 your_image = cv2.imread(your_image_path, cv2.IMREAD_GRAYSCALE)

# Process the image using the process_image function
 processed_image = process_image(your_image)

# Expand dimensions to match the model's input shape
 processed_image = np.expand_dims(processed_image, axis=0)

# Make predictions on your processed image
 prediction = act_model.predict(processed_image)

# Use CTC decoder
 decoded = K.ctc_decode(prediction,
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]

 out = K.get_value(decoded)

# Display the results
 for _, x in enumerate(out):
    print("predicted text = ", end='')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end='')
    
    # Display your processed image
    plt.imshow(your_image, cmap=plt.cm.gray)
    plt.show()

      


In [ ]:
import cv2
import numpy as np
# Load your image from file
for i in range(4):
 your_image_path = "D:\word_segmentation-master\segmented\segment"+ str(i) + ".png"
 your_image = cv2.imread(your_image_path, cv2.IMREAD_GRAYSCALE)

# Process the image using the process_image function
 processed_image = process_image(your_image)

# Expand dimensions to match the model's input shape
 processed_image = np.expand_dims(processed_image, axis=0)

# Make predictions on your processed image
 prediction = act_model.predict(processed_image)

# Use CTC decoder
 decoded = K.ctc_decode(prediction,
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]

 out = K.get_value(decoded)

# Display the results
 for _, x in enumerate(out):
    print("predicted text = ", end='')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end='')
    
    # Display your processed image
    plt.imshow(your_image, cmap=plt.cm.gray)
    plt.show()

      


In [ ]:
plotgraph(epochs, loss, val_loss)

In [ ]:
plotgraph(epochs, acc, val_acc)

In [ ]:
# get best model index
minimum_val_loss = np.min(history.history['val_loss'])
best_model_index = np.where(history.history['val_loss'] == minimum_val_loss)[0][0]

best_loss = str(history.history['loss'][best_model_index])
best_acc = str(history.history['accuracy'][best_model_index])
best_val_loss = str(history.history['val_loss'][best_model_index])
best_val_acc = str(history.history['val_accuracy'][best_model_index])

In [ ]:
print(best_loss)
print(best_acc)
print(best_val_acc)